Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Devon Botney"
COLLABORATORS = "Paul Cantave"

---

In [1]:
#First the encoded data needs to be correctly turned into a sorted CSV (currently one column)
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [2]:
#To get rid of studentID, Country, Question ID, and Keywords
input = "raw_data.csv"
output = "simple_data.csv"
columns = ['Type_of_Answer', 'Question_Level', 'Topic', 'Subtopic']
with open(input, 'r', encoding='utf-8') as infile, open(output, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.DictReader(infile, delimiter=';')
    writer = csv.DictWriter(outfile, fieldnames=columns, delimiter=';')
    writer.writeheader()
    for row in reader:
        filtered = {key: row[key] for key in columns if key in row}
        writer.writerow(filtered)
print(f"Filtered")

Filtered


In [3]:
#Creation of a file that shows the different Topic/Subtopic pairs
input = "simple_data.csv"
output = "topics.csv"

topic_col = 'Topic'
sub_col = 'Subtopic'

unique = set()

with open(input, 'r', encoding='utf-8') as infile:
    reader = csv.DictReader(infile, delimiter=';')
    for row in reader:
        topic = row.get(topic_col)
        sub = row.get(sub_col)
        if topic and sub:
            unique.add((topic, sub))
with open(output, 'w', encoding='utf-8', newline='') as outfile:
    writer = csv.writer(outfile, delimiter=';')
    writer.writerow([topic_col, sub_col])
    writer.writerows(sorted(unique))
print(f"Finished")

Finished


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

file = "simple_data.csv"
data = pd.read_csv(file, delimiter=';')

label_encoders = {}
for column in ['Question_Level', 'Topic', 'Subtopic']:
    label = LabelEncoder()
    data[column] = label.fit_transform(data[column])
    label_encoders[column] = label

x = data[['Question_Level', 'Topic', 'Subtopic']]
y = data['Type_of_Answer']

#Train with seed 42
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)
model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.55


In [6]:
import sqlite3
conn = sqlite3.connect('mathvalues.db')
print(f"Connection was successful.")

Connection was successful.


In [7]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text


file_path = 'simple_data.csv'
df = pd.read_csv(file_path, delimiter=';')


engine = create_engine('sqlite:///mathvalues.db') 


table_name = 'math_tables'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

with engine.connect() as conn:
    result = conn.execute(text("SELECT Topic, 100.0 * SUM(CASE WHEN Type_of_Answer = 1 THEN 1 ELSE 0 END) / COUNT(*) AS percentage FROM math_tables GROUP BY Topic;"))
    for row in result:
        print(row)

('Analytic Geometry', 48.88268156424581)
('Complex Numbers', 45.439189189189186)
('Differential Equations', 53.7037037037037)
('Differentiation', 34.196891191709845)
('Fundamental Mathematics', 46.577017114914426)
('Graph Theory', 58.18181818181818)
('Integration', 44.44444444444444)
('Linear Algebra', 49.02200488997555)
('Numerical Methods', 38.70967741935484)
('Optimization', 38.46153846153846)
('Probability ', 37.5)
('Real Functions of a single variable', 35.36585365853659)
('Set Theory', 64.28571428571429)
('Statistics', 47.94117647058823)


In [8]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text


file_path = 'simple_data.csv'
df = pd.read_csv(file_path, delimiter=';')


engine = create_engine('sqlite:///mathvalues.db') 


table_name = 'math_tables'
df.to_sql(table_name, con=engine, if_exists='replace', index=False)

with engine.connect() as conn:
    result = conn.execute(text("SELECT Topic, 100.0 * SUM(CASE WHEN Type_of_Answer = 1 AND Question_Level = 'Basic' THEN 1 ELSE 0 END) / COUNT(CASE WHEN Question_Level = 'Basic' THEN 1 ELSE NULL END) AS percentage FROM math_tables GROUP BY Topic;"))
    for row in result:
        print(row)
    print("-----------------------")
    result2 = conn.execute(text("SELECT Topic, 100.0 * SUM(CASE WHEN Type_of_Answer = 1 AND Question_Level = 'Advanced' THEN 1 ELSE 0 END) / COUNT(CASE WHEN Question_Level = 'Advanced' THEN 1 ELSE NULL END) AS percentage FROM math_tables GROUP BY Topic;"))
    for row in result2:
        print(row)
    print("-----------------------")
    result3 = conn.execute(text("SELECT Topic, SUM(CASE WHEN Question_Level = 'Advanced' THEN 1 ELSE 0 END) AS Advanced_Questions FROM math_tables GROUP BY Topic;"))
    for row in result3:
        print(row)
    
        

('Analytic Geometry', 51.298701298701296)
('Complex Numbers', 35.97560975609756)
('Differential Equations', 53.77358490566038)
('Differentiation', 33.03249097472924)
('Fundamental Mathematics', 46.35761589403973)
('Graph Theory', 48.57142857142857)
('Integration', 45.6)
('Linear Algebra', 48.83208461877479)
('Numerical Methods', 38.70967741935484)
('Optimization', 39.77272727272727)
('Probability ', 35.23809523809524)
('Real Functions of a single variable', 36.0655737704918)
('Set Theory', 64.28571428571429)
('Statistics', 47.94117647058823)
-----------------------
('Analytic Geometry', 34.0)
('Complex Numbers', 57.196969696969695)
('Differential Equations', 50.0)
('Differentiation', 60.0)
('Fundamental Mathematics', 49.20634920634921)
('Graph Theory', 75.0)
('Integration', 36.8421052631579)
('Linear Algebra', 49.74747474747475)
('Numerical Methods', None)
('Optimization', 0.0)
('Probability ', 47.82608695652174)
('Real Functions of a single variable', 33.333333333333336)
('Set Theory'